# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, string, random

# Конфигурация

In [2]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
torch.cuda.random.manual_seed_all(RANDOM_SEED)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
device

device(type='cuda')

# Загрузка данных

In [4]:
data = pd.read_csv('../../data-parsing/parser-otzovik/reviews.csv')

In [5]:
data.head()

,Name_of_review,Date,Advantages,Disadvantages,Main_text,Overall_assessment,Assortment,Prices,Delivery,Availability,Service,isRecommended
0,недорогой магазин рядом с домом,18.08.2010,"дёшево, удобно",не большой ассортимент,С удовольствием посещаю магазин эконом класса ...,4,4,4,4,5,3,1
1,Закупаемся в Пятерочке,19.08.2010,"цены, выбор, акции","даже не знаю, все-таки не один магазин","Раз в неделю, а может чаще (когда как) мы отпр...",5,3,4,3,5,3,1
2,Хочу поделиться своим горьким опытом работы в ...,20.08.2010,близко,"низкая зп, обман, штрафы, неуважение",Довелось как-то мне однажды от безвыходности у...,1,3,4,1,3,1,0
3,"Ужасный магазин, ужасное обслуживание",16.09.2010,не нашла,"просроченные продукты, невоспитанный персонал,...","В магазин ""Пятерочка"" зареклась ходить за поку...",1,3,3,3,3,1,0
4,Больше не зайду!,27.09.2010,не обнаружено;,сплошные минусы,"Побывала я как то в магазине эконом-класса ""Пя...",1,1,1,1,1,1,0


# Загрузка модели

In [6]:
tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
model = AutoModelForSequenceClassification.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
# problem_type = "single_label_classification"

In [19]:
text = """""Дикси", как "Дикси" Постоянно нужно следить, чтобы товары на полке соответствовали ценникам, хамоватый восточного вида управляющий.. Ну и кассир бабуля, которую покупатели, откроненно раздражают.. Правда,буду честным есть несколько нормальных кассиров и рыжеволосая менеджер хорошая:)"""

list_label = ["персонал магазина",
              "цены товаров",
              "ассортимент товаров",
              "качество товаров",
              "чистота в магазине",
              "расположение магазина"]

# list_label = ["хороший персонал магазина", "плохой персонал магазина",
#               "высокие цены товаров", "низкие цены товаров",
#               "широкий ассортимент товаров", "скудный ассортимент товаров",
#               "высокое качество товаров", "низкое качество товаров",
#               "высокая чистота магазина", "низкая чистота магазина",
#               "удобное расположение магазина", "неудобное расположение магазина"]

# list_label = ["грязный магазин","чистый магазин"]

In [20]:
list_ABC = [x for x in string.ascii_uppercase]

In [21]:
def check_text(model, text, list_label, shuffle=False):
    list_label = [x+'.' if x[-1] != '.' else x for x in list_label]
    list_label_new = list_label + [tokenizer.pad_token]* (20 - len(list_label))
    if shuffle: 
        random.shuffle(list_label_new)
    s_option = ' '.join(['('+list_ABC[i]+') '+list_label_new[i] for i in range(len(list_label_new))])
    text = f'{s_option} {tokenizer.sep_token} {text}'

    model.to(device).eval()
    encoding = tokenizer([text],truncation=True, max_length=512,return_tensors='pt')
    item = {key: val.to(device) for key, val in encoding.items()}
    logits = model(**item).logits

    logits = logits if shuffle else logits[:,0:len(list_label)]
    probs = torch.nn.functional.softmax(logits, dim = -1).tolist()
#     predictions = torch.argmax(logits, dim=-1).item()
    predictions = torch.IntTensor(np.arange(len(list_label)))
    probabilities = [round(x,5) for x in probs[0]]
    
    new_probabilities, new_list_label = zip(*[(b, a) for b, a in sorted(zip(probabilities, list_label))])
    for i in range(len(new_list_label)-1,-1,-1):
        print(new_list_label[i][:-1], ' - ', round(new_probabilities[i],2))

#     print(f'prediction:    {predictions} => ({list_ABC[predictions]}) {list_label_new[predictions]}')
#     print(f'probability:   {round(probabilities[predictions]*100,2)}%')

In [22]:
%%time
check_text(model, text, list_label)

персонал магазина  -  0.34
чистота в магазине  -  0.31
качество товаров  -  0.11
цены товаров  -  0.1
расположение магазина  -  0.1
ассортимент товаров  -  0.04
CPU times: total: 62.5 ms
Wall time: 90.5 ms


In [5]:
!nvidia-smi

Thu Sep  7 16:58:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   65C    P0    57W / 115W |   1892MiB /  8192MiB |     39%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------